In [ ]:
pip install torch transformers pandas sacrebleu nltk

In [ ]:
import pandas as pd
from translator import NMTTranslator
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
class TranslationEvaluator:
    def __init__(self):
        self.smoothing = SmoothingFunction()
    
    def calculate_bleu(self, reference, candidate):
        ref_tokens = nltk.word_tokenize(reference.lower())
        cand_tokens = nltk.word_tokenize(candidate.lower())
        score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=self.smoothing.method1)
        return round(score * 100, 2)
    
    def calculate_meteor(self, reference, candidate):
        score = meteor_score([reference], candidate)
        return round(score * 100, 2)

def run_evaluation(translator, test_cases):
    evaluator = TranslationEvaluator()
    results = []
    for case in test_cases:
        result = translator.translate(case['source'], case['src_lang'], case['tgt_lang'])
        if 'error' not in result:
            pred_text = result['translation']
            bleu = evaluator.calculate_bleu(case['reference'], pred_text)
            meteor = evaluator.calculate_meteor(case['reference'], pred_text)
            results.append({
                'source': case['source'],
                'reference': case['reference'],
                'prediction': pred_text,
                'bleu': bleu,
                'meteor': meteor
            })
    return pd.DataFrame(results)

In [ ]:
nmt = NMTTranslator()

evaluation_pairs = [
    {'source': 'Hello, how are you?', 'reference': 'नमस्ते, आप कैसे हैं?', 'src_lang': 'en', 'tgt_lang': 'hi'},
    {'source': 'Good morning', 'reference': 'सुप्रभात', 'src_lang': 'en', 'tgt_lang': 'hi'},
    {'source': 'Thank you', 'reference': 'धन्यवाद', 'src_lang': 'en', 'tgt_lang': 'hi'},
    {'source': 'नमस्ते', 'reference': 'Hello', 'src_lang': 'hi', 'tgt_lang': 'en'}
]

evaluation_df = run_evaluation(nmt, evaluation_pairs)
print(evaluation_df)